In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

In [21]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.2):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [22]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, emb_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb  = layers.Embedding(input_dim = vocab_size, 
                                           output_dim = emb_dim, 
                                           embeddings_initializer=keras.initializers.Constant(matrix),
                                           trainable=True)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=emb_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [25]:
stfold = StratifiedKFold(n_splits=5, shuffle=True)
train_l = list()
val_l = list()
test_l = list()
maxlen = 100
num_heads = 8
ff_dim = 32 

cnt = 0
for train_index, test_index in stfold.split(data['comment'], data['label']):
    cnt += 1
    train = data.iloc[train_index]
    train_x, train_y = train['comment'], train['label']
    tmp = data.iloc[test_index]

    val, test = train_test_split(tmp, test_size=0.5, stratify=tmp['label'])    
    val_x, val_y = val['comment'], val['label']
    test_x, test_y = test['comment'], test['label']    
    
    train_x = vectorizer(np.array([[s] for s in train_x])).numpy()    
    val_x = vectorizer(np.array([[s] for s in val_x])).numpy()
    test_x = vectorizer(np.array([[s] for s in test_x])).numpy()

    train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
    val_x = keras.preprocessing.sequence.pad_sequences(val_x, maxlen=maxlen)
    test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
    

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, emb_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(emb_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(20, activation="relu")(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)


    model = keras.Model(inputs=inputs, outputs=outputs)
    model_path = "transformer_models/{itr:04}".format(itr=cnt)
    checkpoint_path = model_path + "-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    !mkdir $model_path
    
    model.compile("adam", "binary_crossentropy", metrics=["accuracy", 
                                                          tf.keras.metrics.AUC(name='auc')])
    
  
    #create checkpoint to save model
    #with best validation loss
    model.save_weights(checkpoint_path.format(epoch=0))

    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1,
            save_weights_only=True, save_best_only=True, mode='auto')
    
    history = model.fit(
                        train_x, train_y, 
                        batch_size=128, epochs=10, 
                        validation_data=(val_x, val_y),
                        callbacks = [checkpoint]
                        )
    
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    train_eval = model.evaluate(train_x, train_y)
    val_eval = model.evaluate(val_x, val_y)
    test_eval = model.evaluate(test_x, test_y)
    
    test_l.append(test_eval)
    val_l.append(val_eval)
    train_l.append(train_eval)

mkdir: cannot create directory ‘transformer_models/0001’: No such file or directory
Epoch 1/10
63/63 [==============================] - 327s 5s/step - loss: 0.8357 - accuracy: 0.5180 - auc: 0.5238 - val_loss: 0.7864 - val_accuracy: 0.5380 - val_auc: 0.7054

Epoch 00001: val_loss improved from inf to 0.78639, saving model to transformer_models/0001-0001.ckpt
Epoch 2/10
63/63 [==============================] - 322s 5s/step - loss: 0.4359 - accuracy: 0.7935 - auc: 0.8792 - val_loss: 0.4540 - val_accuracy: 0.7950 - val_auc: 0.8826

Epoch 00002: val_loss improved from 0.78639 to 0.45397, saving model to transformer_models/0001-0002.ckpt
Epoch 3/10
63/63 [==============================] - 323s 5s/step - loss: 0.1231 - accuracy: 0.9604 - auc: 0.9903 - val_loss: 0.5656 - val_accuracy: 0.8380 - val_auc: 0.9108

Epoch 00003: val_loss did not improve from 0.45397
Epoch 4/10
63/63 [==============================] - 328s 5s/step - loss: 0.0312 - accuracy: 0.9935 - auc: 0.9987 - val_loss: 1.0799 - v

In [26]:
test_l = np.array(test_l)
val_l = np.array(val_l)
train_l = np.array(train_l)

print("test avg loss: ", np.mean(test_l[:, 0]), "+/-" ,np.std(test_l[:, 0]))
print("test avg acc: ", np.mean(test_l[:, 1]),  "+/-" ,np.std(test_l[:, 1]))
print("test avg auc: ", np.mean(test_l[:, 2]),  "+/-" ,np.std(test_l[:, 2]))
print('\n')


print("val avg loss: ", np.mean(val_l[:, 0]), "+/-" ,np.std(val_l[:, 0]))
print("val avg acc: ", np.mean(val_l[:, 1]),  "+/-" ,np.std(val_l[:, 1]))
print("val avg auc: ", np.mean(val_l[:, 2]),  "+/-" ,np.std(val_l[:, 2]))
print('\n')


print("train avg loss: ", np.mean(train_l[:, 0]), "+/-" ,np.std(train_l[:, 0]))
print("train avg acc: ", np.mean(train_l[:, 1]),  "+/-" ,np.std(train_l[:, 1]))
print("train avg auc: ", np.mean(train_l[:, 2]),  "+/-" ,np.std(train_l[:, 2]))

test avg loss:  0.5136931657791137 +/- 0.06994241723575245
test avg acc:  0.8111999988555908 +/- 0.02382770826111824
test avg auc:  0.8985428810119629 +/- 0.015285996380062995


val avg loss:  0.5257814049720764 +/- 0.10599973988018306
val avg acc:  0.8116000175476075 +/- 0.030156914620622366
val avg auc:  0.8952931880950927 +/- 0.010714450573773621


train avg loss:  0.049974911380559206 +/- 0.04680226783502984
train avg acc:  0.9892250061035156 +/- 0.010128676590453767
train avg auc:  0.9987109541893006 +/- 0.0017073192278102673
